# 08 - Navigatie en app-indeling

> tags: []

Learn to set up a parent layout and navigate between child views.


We hebben de documentatie gereorganiseerd om u een meer gestroomlijnde navigatiestructuur te bieden

1.  [Documentatie](https://vaadin.com/docs/latest/)
2.  [Tutorial](https://vaadin.com/docs/latest/tutorial)
3.  Navigatie en app-indeling

-   [Weergaveroutes definiëren](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#defining-view-routes)
-   [De bovenliggende lay-out maken](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#creating-the-parent-layout)
-   [Een dashboardweergave maken](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#creating-a-dashboard-view)
-   [De dashboardweergave toevoegen aan de zijbalk van de hoofdlay-out](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#add-the-dashboard-view-to-the-main-layout-sidebar)

Tot nu toe hebt u in deze zelfstudiereeks een CRM-toepassing gebouwd voor het vermelden en bewerken van contactpersonen. In dit hoofdstuk voegt u een dashboardweergave toe aan de toepassing. U voegt ook een responsieve toepassingslay-out toe, met een koptekst en een navigatiezijbalk die op kleine schermen kan worden ingeschakeld.

![Toepassingslay-out met een koptekst en een zijbalk](https://vaadin.com/docs/latest/static/b2306f4a0b9f0cc81920c8cb49429792/03979/app-layout.png)

## [](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#defining-view-routes)Weergaveroutes definiëren

U kunt van elke Vaadin-component een navigatiedoel maken door een annotatie toe te voegen. Routes kunnen worden genest door de bovenliggende lay-out te definiëren in de annotatie: .`@Route("<path>")``@Route(value = "list", layout=MainLayout.class)`

## [](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#creating-the-parent-layout)De bovenliggende lay-out maken

De aanvraag moet het volgende bevatten:

-   een gedeelde bovenliggende indeling met twee onderliggende weergaven:
    
    `MainLayout`: App Layout met header en navigatie:
    
    1.  `ListView`: de standaardweergave, toegewezen aan .`""`
        
    2.  `DashboardView`: toegewezen aan .`"dashboard"`
        
    
-   een responsieve applicatie lay-out en navigatie links.
    

Begin met het maken van een nieuwe Java-klasse met de naam in het pakket met de volgende inhoud. Dit is de gedeelde bovenliggende lay-out van beide weergaven in de toepassing.`MainLayout``views`

```
package com.example.application.views;

import com.example.application.views.list.ListView;
import com.vaadin.flow.component.applayout.AppLayout;
import com.vaadin.flow.component.applayout.DrawerToggle;
import com.vaadin.flow.component.html.H1;
import com.vaadin.flow.component.orderedlayout.FlexComponent;
import com.vaadin.flow.component.orderedlayout.HorizontalLayout;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.router.HighlightConditions;
import com.vaadin.flow.router.RouterLink;
import com.vaadin.flow.server.PWA;

public class MainLayout extends AppLayout { 1

    public MainLayout() {
        createHeader();
        createDrawer();
    }

    private void createHeader() {
        H1 logo = new H1("Vaadin CRM");
        logo.addClassNames("text-l", "m-m");

        HorizontalLayout header = new HorizontalLayout(
          new DrawerToggle(), 2
          logo
        );

        header.setDefaultVerticalComponentAlignment(FlexComponent.Alignment.CENTER); 3
        header.setWidth("100%");
        header.addClassNames("py-0", "px-m");

        addToNavbar(header); 4

    }

    private void createDrawer() {
        RouterLink listLink = new RouterLink("List", ListView.class); 5
        listLink.setHighlightCondition(HighlightConditions.sameLocation()); 6

        addToDrawer(new VerticalLayout( 7
            listLink
        ));
    }
}
```

1.  `AppLayout` is een Vaadin lay-out met een koptekst en een responsieve lade.
    
2.  `DrawerToggle` is een menuknop die de zichtbaarheid van de zijbalk schakelt.
    
3.  Centreert de componenten langs de verticale as.`header`
    
4.  Hiermee voegt u de lay-out toe aan de navigatiebalk van de toepassingslay-out, het gedeelte boven aan het scherm.`header`
    
5.  Maakt een met de tekst "Lijst" en als de bestemmingsweergave.`RouterLink``ListView.class`
    
6.  Hiermee stelt u in om te voorkomen dat de koppeling wordt gemarkeerd voor gedeeltelijke routeovereenkomsten. (Technisch gezien begint elke route met een lege route, dus zonder dit zou deze altijd als actief worden weergegeven, ook al is de gebruiker niet in beeld.)`setHighlightCondition(HighlightConditions.sameLocation())`
    
7.  Verpakt de link in een en voegt deze toe aan de lade van de 'AppLayout.`VerticalLayout`
    

Voeg de volgende CSS toe aan frontend/themes/flowcrmtutorial/styles.css om de geselecteerde link te markeren.

```
a[highlight] {
    font-weight: bold;
    text-decoration: underline;
}
```

Werk ten slotte in , de toewijzing bij om de nieuwe .`ListView``@Route``MainLayout`

```
@Route(value="", layout = MainLayout.class) 1
@PageTitle("Contacts | Vaadin CRM")
public class ListView extends VerticalLayout {
     ...
}
```

1.  `ListView` komt nog steeds overeen met het lege pad, maar wordt nu gebruikt als bovenliggend pad.`MainLayout`
    

Voer de toepassing uit. U zou nu een koptekst en een zijbalk in de lijstweergave moeten zien.

![Toepassing met lijstweergave in de zijbalk](https://vaadin.com/docs/latest/static/b06234c0f1030d4c3721eff1024e00bc/03979/app-layout-list-view.png)

## [](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#creating-a-dashboard-view)Een dashboardweergave maken

Maak vervolgens een nieuwe dashboardweergave. Deze weergave toont statistieken: het aantal contacten in het systeem en een cirkeldiagram van het aantal contacten per bedrijf.

![Dashboardweergave](https://vaadin.com/docs/latest/static/a2b8d0ae3b8b0652e3807ab7eec581bb/03979/dashboard-view.png)

Maak een nieuwe Java-klasse met de naam in het pakket met de volgende inhoud:`DashboardView``views`

```
package com.example.application.views;

import com.example.application.data.service.CrmService;
import com.vaadin.flow.component.Component;
import com.vaadin.flow.component.charts.Chart;
import com.vaadin.flow.component.charts.model.ChartType;
import com.vaadin.flow.component.charts.model.DataSeries;
import com.vaadin.flow.component.charts.model.DataSeriesItem;
import com.vaadin.flow.component.html.Span;
import com.vaadin.flow.component.orderedlayout.VerticalLayout;
import com.vaadin.flow.router.PageTitle;
import com.vaadin.flow.router.Route;

@Route(value = "dashboard", layout = MainLayout.class) 1
@PageTitle("Dashboard | Vaadin CRM")
public class DashboardView extends VerticalLayout {
    private final CrmService service;

    public DashboardView(CrmService service) { 2
        this.service = service;
        addClassName("dashboard-view");
        setDefaultHorizontalComponentAlignment(Alignment.CENTER); 3
        add(getContactStats(), getCompaniesChart());
    }

    private Component getContactStats() {
        Span stats = new Span(service.countContacts() + " contacts"); 3
        stats.addClassNames("text-xl", "mt-m");
        return stats;
    }

    private Chart getCompaniesChart() {
        Chart chart = new Chart(ChartType.PIE); 4

        DataSeries dataSeries = new DataSeries();
        service.findAllCompanies().forEach(company ->
            dataSeries.add(new DataSeriesItem(company.getName(), company.getEmployeeCount()))); 5
        chart.getConfiguration().setSeries(dataSeries);
        return chart;
    }
}
```

1.  `DashboardView` wordt toegewezen aan het pad en gebruikt als bovenliggende lay-out.`"dashboard"``MainLayout`
    
2.  Neemt als constructorparameter en slaat deze op als een veld.`CrmService`
    
3.  Centreert de inhoud van de lay-out.
    
4.  Belt de service om het aantal contacten op te halen.
    
5.  Roept de service aan om alle bedrijven te krijgen en maakt vervolgens een voor elk bedrijf, met de bedrijfsnaam en het aantal werknemers.`DataSeriesItem`
    

<table><tbody><tr><td _msthidden="1"><p>Note</p></td><td><p>Vaadin Charts is een commerciële componentenset</p><p _msthash="2705771" _msttexthash="50204869"><a href="https://vaadin.com/docs/latest/components/charts" _istranslated="1">Vaadin Charts</a> is een verzameling gegevensvisualisatiecomponenten die deel uitmaakt van het <a href="https://vaadin.com/pricing" _istranslated="1">Vaadin Pro-abonnement</a>. Vaadin Charts wordt geleverd met een gratis proefversie die u in de browser kunt activeren. Alle Vaadin Pro tools en componenten zijn gratis voor studenten via het <a href="https://education.github.com/pack?utm_source=github+vaadin" _istranslated="1">GitHub Student Developer Pack</a>.</p></td></tr></tbody></table>

Open Bedrijf.java en voeg het volgende veld en getter toe om het aantal werknemers te krijgen zonder alle entiteiten te hoeven ophalen.

```
@Formula("(select count(c.id) from Contact c where c.company_id = id)") 1
private int employeeCount;

public int getEmployeeCount(){
    return employeeCount;
}
```

1.  De Formule krijgt de telling van werknemers zonder dat alle werknemers hoeven te worden opgehaald.
    

## [](https://vaadin.com/docs/latest/tutorial/navigation-and-layouts/#add-the-dashboard-view-to-the-main-layout-sidebar)De dashboardweergave toevoegen aan de zijbalk van de hoofdlay-out

Voeg een navigatiekoppeling toe aan in de lade:`DashboardView``MainLayout`

```
private void createDrawer() {
    RouterLink listLink = new RouterLink("List", ListView.class);
    listLink.setHighlightCondition(HighlightConditions.sameLocation());

    addToDrawer(new VerticalLayout(
        listLink,
        new RouterLink("Dashboard", DashboardView.class)
    ));
}
```

Build and run the application. You should now be able to navigate to the dashboard view and see stats on your CRM contacts. If you want to, go ahead and add or remove contacts in the list view to see that the dashboard reflects your changes.

![Complete dashboard view](https://vaadin.com/docs/latest/static/aafc4e177cf3d859c7c16bcf43d41ce2/03979/dashboard-completed.png)

In the next chapter, you will secure the application by adding a log-in screen.

52AFFD31-EA40-4AEF-B60F-E3BB6E5A8379

[TutorialHandling View State](https://vaadin.com/docs/latest/tutorial/handling-view-state)[TutorialLogin and Authentication](https://vaadin.com/docs/latest/tutorial/login-and-authentication)

**Was this page helpful?**  
Leave a comment or a question below. You can also join the [chat on Discord](https://discord.gg/MYFq5RTbBn) or [ask questions on StackOverflow](https://stackoverflow.com/questions/tagged/vaadin).
